In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
dataset=pd.read_csv("Social_Network_Ads.csv")

In [3]:
dataset

,User ID,Gender,Age,EstimatedSalary,Purchased
0,15624510,Male,19,19000,0
1,15810944,Male,35,20000,0
2,15668575,Female,26,43000,0
3,15603246,Female,27,57000,0
4,15804002,Male,19,76000,0
...,...,...,...,...,...
395,15691863,Female,46,41000,1
396,15706071,Male,51,23000,1
397,15654296,Female,50,20000,1
398,15755018,Male,36,33000,0


In [4]:
dataset["Purchased"].value_counts()

Purchased
0    257
1    143
Name: count, dtype: int64

In [5]:
dataset.columns


Index(['User ID', 'Gender', 'Age', 'EstimatedSalary', 'Purchased'], dtype='object')

In [6]:
dataset=pd.get_dummies(dataset,dtype=int,drop_first=True)

In [7]:
dataset


,User ID,Age,EstimatedSalary,Purchased,Gender_Male
0,15624510,19,19000,0,1
1,15810944,35,20000,0,1
2,15668575,26,43000,0,0
3,15603246,27,57000,0,0
4,15804002,19,76000,0,1
...,...,...,...,...,...
395,15691863,46,41000,1,0
396,15706071,51,23000,1,1
397,15654296,50,20000,1,0
398,15755018,36,33000,0,1


In [8]:
dataset.columns

Index(['User ID', 'Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')

In [9]:
independent=dataset[['Age', 'EstimatedSalary','Gender_Male']]
dependent=dataset['Purchased']

In [10]:
from sklearn.model_selection import train_test_split
#calling the parameterized function train_test_split 
x_train,x_test,y_train,y_test=train_test_split(independent, dependent, test_size=1/3, random_state=0)

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [20]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV

param_grid = {'kernel':['linear','rbf','poly','sigmoid'],
             'gamma':['auto','scale'],
             'C':[10,100,1000,2000,3000]} 
grid = GridSearchCV(SVC(probability=True), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(x_train, y_train) 
 


Fitting 5 folds for each of 40 candidates, totalling 200 fits


GridSearchCV(estimator=SVC(probability=True), n_jobs=-1,
             param_grid={'C': [10, 100, 1000, 2000, 3000],
                         'gamma': ['auto', 'scale'],
                         'kernel': ['linear', 'rbf', 'poly', 'sigmoid']},
             scoring='f1_weighted', verbose=3)

In [21]:
# print best parameter after tuning 
#print(grid.best_params_) 
re=grid.cv_results_
#print(re)
grid_predictions = grid.predict(x_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, grid_predictions)

In [14]:
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'C': 100, 'gamma': 'auto', 'kernel': 'rbf'}: 0.9100355779243318


In [22]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[80  5]
 [ 7 42]]


In [23]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.92      0.94      0.93        85
           1       0.89      0.86      0.88        49

    accuracy                           0.91       134
   macro avg       0.91      0.90      0.90       134
weighted avg       0.91      0.91      0.91       134



In [27]:
x_test
a=grid.predict_proba(x_test)
print(a)

[[0.84077506 0.15922494]
 [0.97609613 0.02390387]
 [0.86792278 0.13207722]
 [0.94152766 0.05847234]
 [0.99104882 0.00895118]
 [0.9393936  0.0606064 ]
 [0.99845845 0.00154155]
 [0.19117498 0.80882502]
 [0.99648913 0.00351087]
 [0.52895828 0.47104172]
 [0.95192732 0.04807268]
 [0.99623087 0.00376913]
 [0.91323334 0.08676666]
 [0.79080241 0.20919759]
 [0.98799106 0.01200894]
 [0.26131208 0.73868792]
 [0.8172263  0.1827737 ]
 [0.98985497 0.01014503]
 [0.28093529 0.71906471]
 [0.78215084 0.21784916]
 [0.99183712 0.00816288]
 [0.24095889 0.75904111]
 [0.93213668 0.06786332]
 [0.06479902 0.93520098]
 [0.99705023 0.00294977]
 [0.24602472 0.75397528]
 [0.93956156 0.06043844]
 [0.93434979 0.06565021]
 [0.97221176 0.02778824]
 [0.79143654 0.20856346]
 [0.98336309 0.01663691]
 [0.7839756  0.2160244 ]
 [0.22451018 0.77548982]
 [0.90894784 0.09105216]
 [0.98679614 0.01320386]
 [0.98714448 0.01285552]
 [0.98855598 0.01144402]
 [0.99463799 0.00536201]
 [0.9981058  0.0018942 ]
 [0.2142149  0.7857851 ]


In [32]:
roc_auc_score(y_test,grid.predict_proba(x_test)[:,1])

0.9539015606242497

In [ ]:
table=pd.DataFrame.from_dict(re)
table